# Dynamic optimization

## Index
1. starting from single shots
2. coding raw videos in AVC, HEVC, VP9 (in seguito ed eventualmente AV1 e VVC) using:
    - ffmpeg library for AVC
    - TODO
    
3. script per generazione automatica di codifiche multiple, in python, includendo anche risoluzioni fino a 4k (sulle 10 risoluzioni proposte dal paper, selezionarne 5 e aggiungerne 2 oltre l'HD) con convex hull RD dei singoli shot e visualizzazione in MATLAB

4. valutazione risultati con PSNR, MSSIM e soprattutto VMAF e comparazione per contenuto dei singoli shot

5. memorizzazione risultati su csv

6. divisione video in spezzoni con ffmpeg

7. script python per generazione dell'intero video codificato seguendo i passi descritti finora per ogni shot e con convex hull generale; test su 3 titoli: ritmo lento, azione e animazione

8. valutazione risultati come punto e. anche in base alla durata degli shot

In [1]:
import ffmpeg
!ffprobe temp/in_to_tree_444_720p50.y4m

ffprobe version 4.2.4-1ubuntu0.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-